In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
! pip install -q peft  accelerate bitsandbytes safetensors

In [ ]:
!pip install --upgrade --no-cache-dir auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

In [ ]:
!pip install optimum

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
# adapters_name = "Huss9/Mistral-7B-LegalNER"
adapters_name = "/content/drive/MyDrive/MAI/Q3/HLE/Project/Models/Mistral_finetuned_ner/checkpoint-400"
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ" #"mistralai/Mistral-7B-Instruct-v0.1"


device = "cuda" # the device to load the model onto


In [ ]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='cuda'
)

In [ ]:
model = PeftModel.from_pretrained(model, adapters_name)
#model = model.merge_and_unload()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

In [ ]:
model.to(device)

In [ ]:
import pandas as pd
# path = "/content/drive/MyDrive/Inlab/Datasets/Prova_Ner/"
path = "/content/drive/MyDrive/MAI/Q3/HLE/Project/Data/Finetuning/Mistral/First Format/"
train_data = pd.read_csv(path+"train.csv")
dev_data = pd.read_csv(path+"dev.csv")
test_data = pd.read_csv(path+"test.csv")

In [ ]:
i = 100
sentence = test_data['sentence'].iloc[i]
groundtruh = test_data['raw_entities'].iloc[i]
prompt = test_data['text'].iloc[i]
print(sentence)
print(groundtruh)
print(prompt)

In [ ]:
instruction = "You are solving the NER problem. You have to extract from the text, words related to each of the following categories: CASE_NUMBER, COURT, DATE, GPE, JUDGE, LAWYER, ORG, OTHER_PERSON, PETITIONER, PRECEDENT, PROVISION, RESPONDENT, STATUTE, WITNESS. Extract them exactly as they are in the text (Don't format them). Your output always should be a dictionary, if there aren't any entities in the text, just return an empty dictionary once."
sentence = test_data['sentence'].iloc[i]
text = f"""<s>[INST] {instruction} Find the entities in the following text: {sentence} [/INST]\n"""

In [ ]:
from transformers import GenerationConfig
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=200,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)